# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

This a classification based supervised learning problem, since the goal of this project is to only predict if the student will need early intervention or not, which are basically 2 seperate classes into which the data has to be seperated into. 


## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns)-1 #subtracting 1 to ignore the 'passed' column.

# TODO: Calculate passing students
n_passed = 0;
for i in student_data['passed']:
    if i == 'yes':
        n_passed+=1

# TODO: Calculate failing students
n_failed = n_students-n_passed

# TODO: Calculate graduation rate
grad_rate = (n_passed*100.0)/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)

print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
X_all=X_all.values

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [26]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import StratifiedShuffleSplit
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above

sss = StratifiedShuffleSplit(y_all,test_size=num_test/395.0, random_state=1)
for train_index, test_index in sss:
    X_train, X_test = X_all[train_index], X_all[test_index]
    y_train, y_test = y_all[train_index], y_all[test_index]
# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

I feel that the 3 most appropriate supervised learning models for this problem are 
* K-Nearest Neighbors
* Support vector machines
* Bagging Classifier

### 1. K-Nearest Neighbors:
Finds the mean (weighted) of the k nearest datapoints to the query.

This supervised learning model is used in some forms of predictions in the real world, one of them being text mining - as stated in reasearch papers like this one : [`KNN based Machine Learning Approach for Text and Document Mining`](http://www.sersc.org/journals/IJDTA/vol7_no1/6.pdf). Another wide application of this model is in economic forcasting.
 ##### Strengths:
   - As this is a Instance based learning model, it has almost 0 training time for the data, as all the computation is done only when a query is passed.
   - This is also known as a lazy learning model, as it approximates a value for a query locally for k nearest terms, instead of using the entire dataset. 
   - This model will be extremely useful if the training data is changed constantly, because it does all the predictions only during the time of a query and not everytime the training dataset is updated.
   - Outliers are always ignored while predicting( provided k!=n), as it will almost always be the farthest point from the query, hence tending to give pretty accurate predictions.
   
##### Weaknesses:
- As this is an instance based learning model, querying on this dataset will take considerably longer when compared to any other non-instance based learning models, which will be a problem if queries are passed very often.
- Also, as this approach is based on computing during a query, the model does not actually learn anything, and hence cannot generalize the given data in any way.

This model is one of the best, if not the best candidate, as based on the input data, we need to figure out if a student needs early intervention or not. What better way to do this than to compare this student with other students who have similar features and whether or not these similar students ended up graduating or not. 


### 2. Support Vector Machines:
Finds a line that linearly seperates the given data via kernels.

The SVM learning model have proven to be very accurate when it comes to certain real world applications like Text Categorization ['Text Categorization with Support Vector
Machines'](http://web.cs.iastate.edu/~jtian/cs573/Papers/Joachims-ECML-98.pdf)

##### Strengths:
 - It scales relatively well to high dimensional data when compared to other learning models.
 - We can explicitly control the complexity of the model.
 - The use of kernels is a great advantage as we can mould the dataset into various form of our own to find the best fit. 
 
##### Weaknesses:
 - A poor choice of a kernel will result in high error rates, and hence for a SVM to perform optimally, we need to know which would be the best kernel to be using for a given dataset.
 - Training a dataset takes longer when a more complex kernel is used, and this could be a hinderance when complex models are being procesed
 
This model is a very good choice, as it converts the training data into a linearly seperable model and find the best way of classifying the data into 2 parts, and hence should have very accurate predictions for the unseen data.


### 3. Ensemble Classifier: Bagging Classifier
If a dataset of x datapoints is given, it draws x instances from which the model learns, and this can be repeated several times and put together to get an accurate dataset.

This classifier is used in various fields of research, and there are numerous articles which show its wide range of applications: ['ensemble modeling for nosiheptide fermentation process']('http://fulltext.study/preview/pdf/1181281.pdf')

##### Strengths:
 - it is an excellent model to handle data of mixed type.
 - It tends to keep the variance as low as possible. 
 
##### Weaknesses:
- Increased computation time with increase in the size of the dataset as it creates a large number of instances and learns from them.
- Also, it requires a lot of memory to compute these instances. 
 
As this model always tends to maintain a low f1_score, and with the numerous amounts of instances it creates, it should be classifying the data pretty accurately, and hence would also be one of the best models to choose for this type of classification.



### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [27]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [30]:
# TODO: Import the three supervised learning models from sklearn
#Model A
from sklearn.neighbors import KNeighborsClassifier
#Model B
from sklearn.ensemble import BaggingClassifier
#Model C
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = KNeighborsClassifier()
clf_B = BaggingClassifier()
clf_C = SVC()

# TODO: Set up the training set sizes
for i in [100,200]:
    sss = StratifiedShuffleSplit(y_all, train_size=i/395.0, random_state=1)
    for train_index, test_index in sss:
        X_train[:i],null = X_all[train_index], X_all[test_index]
        y_train[:i],null = y_all[train_index], y_all[test_index]    


X_train_300 = X_train
y_train_300 = y_train

#Model A

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    for size in [100, 200, 300]:
        train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
        print '\n',
    print'---------------------'
        

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8571.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7862.

Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8729.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8175.

Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0010 seconds
Made predictions in 0.0050 seconds.
F1 score for training set: 0.8847.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.8000.

---------------------
Training a BaggingClassifier using a training set size of 100. . .
Trained model in 0.0150 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8296.

Training a Baggin

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

**K-Nearest Neighbors**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0010          |         0.0010         |     0.8571       |      0.7862     |
| 200               |         0.0010          |         0.0010         |     0.8729       |      0.8175     |
| 300               |         0.0010          |         0.0020         |     0.8847       |      0.8000     |
  
**Ensemble Bagging Classifier **

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0140              |         0.0010         |      1.0000      |     0.8296      |
| 200               |     0.0160              |         0.0010         |      0.9926      |     0.8209      |
| 300               |     0.0200              |         0.0010         |      0.9899      |     0.7778      |

**Support Vector Machines**

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0010          |         0.0010         |     0.9189       |      0.8243     |
| 200               |         0.0040          |         0.0010         |     0.9010       |      0.8356     |
| 300               |         0.0070          |         0.0020         |     0.9002       |      0.8276     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

Based on the data in the table above, I have chosen the K nearest neighbors classifier as the best model and will perform model tuning using this classifier. 
I have chosen the above said model because of the following reasons:
1. This model has a way lesser training time when compared to other classifiers, and the training time appears to be constant (or a very low increase with respect to increase in data), where as the in the support vector machines classifier, the training time seems to be increasing exponentially when more training data is provided. The bagging classifier seems to also be taking a lot of time to Choosing this classifier will ensure results are obtained without much delay. Considering the fact that this school data base will be updated regularly as more student graduate, we will need to maintain a low training time.
2. This model also has a high f1 score even before tuning the data, and hence after tuning, we can expect a f1 score of almost 1, which therefore will make predictions almost always accurate.
3. Since this is an instance based learning model, the prediction time is a little high. As this is a very small amount of time, which seems to be a constant, it shouldnt be a major disadvantage when compared to the other training models.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

Based on the student data that was provided, i split it, using 300 students to give the algorithm a basic idea on how to predict, and i have used the rest 95 students to test how well the algorithm is functioning. From the information above, we notice that the f1_score, which is a rating given based on the accuracy, which gives a value between 0 and 1(where 0 is a very bad rating, and 1 is the best rating), is really high for the chosen model, and hence have decided to use this model. 
This model finds 'k' students whose characteristics such as traveltime, studytime, failures, their absences are the most similar to the student in question. This algorithm then finds out who of these 'k' students have passed. If more than 50% of these 'k' students have passed, the model will predict that the student in question will also pass, else the algorithm will predict that he/she has failed.
This model makes the most sense to be applied, as students with many similar characteristics tend to perform quite similarly.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [34]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer
# TODO: Create the parameters list you wish to tune
parameters = {'n_neighbors':range(1,200)}

# TODO: Initialize the classifier
clf = KNeighborsClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(clf)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(clf, parameters)


# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0020 seconds.
Tuned model has a training F1 score of 1.0000.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.8462.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

The chosen tuned model's training F<sub>1</sub> score is 1.0000, and testing F<sub>1</sub> score is 0.8462, while the untuned model's training F<sub>1</sub> score is 0.8847, and testing F<sub>1</sub> score is 0.8000.
By letting GridSearchCV find the best value of 'k' in the KNN model, we could increase the accuracy of predictions of the testing dataset by around 5.78% with respect to the untuned model which takes the default value for k.  

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.